In [5]:
# usage: `python ~/Desktop/crop.py square_found.csv`
# assumes that images in current directory will be found and are the ones
# cropped into the output folder: cropped
# square_found.csv is generated by contours.py

import cv2
import numpy as np
from matplotlib import pyplot as plt
import sys
import math
import pdb
import random as ra
import csv
import json
from util import sort_coords
from geom import skew
import os
import pandas as pd
import random
import imutils

In [7]:
reader = pd.read_csv("square_found.csv", index_col = False) #index_row = False 
reader['id'] = [random.randint(0,1000) for x in range(reader.shape[0])]
#img_name,json_arr,s_height,s_width = row
#df=reader.iloc[-1:,:]
reader.head(6)


,img_name,json_arr,s_height,s_width,id
0,output 8 final.jpg,"[[150, 101], [48, 4243], [2791, 4334], [2905, ...",4500,3000,196
1,18_18.jpeg,"[[112, 108], [171, 2923], [4405, 2837], [4327,...",3000,4500,518
2,output 4 final.jpg,"[[334, 198], [335, 2775], [4160, 2795], [4134,...",3000,4500,812
3,image.jpg,"[[334, 198], [335, 2775], [4160, 2795], [4134,...",3000,4500,291
4,boy.jpg,"[[77, 66], [33, 1135], [748, 1170], [801, 101]]",1280,850,624
5,family.jpg,"[[149, 106], [208, 2900], [4379, 2847], [4302,...",3000,4500,293


In [8]:
img_name = reader.iloc[-1,0]
json_arr = reader.iloc[-1,1]
s_height = reader.iloc[-1,2]
s_width = reader.iloc[-1,3]

print(img_name)
print(json_arr)
print(s_height)
print(s_width)

family.jpg
[[149, 106], [208, 2900], [4379, 2847], [4302, 75]]
3000
4500


In [9]:
#filename = img_source_dir + '/' + img_name
s_height, s_width = [(s_height), (s_width)]
landscape = s_width > s_height
img = cv2.imread("")
#img = imutils.resize(img, width = 800)
l_height,l_width = img.shape[0:2]
#json_arr = int(json_arr)
raw_coords = json.loads(json_arr)
# raw_coords could be in any order, sort them...
center_x = sum([p[0] for p in raw_coords]) / 4
center_y = sum([p[1] for p in raw_coords]) / 4
tl,bl,br,tr = sort_coords(raw_coords)
        
# compute skew
sides = [[tl[::-1], tr[::-1]], [tr, br], [br[::-1], bl[::-1]], [bl, tl]]
avg_skew = sum([abs(skew(*s)) for s in sides]) / 4
rotate_right = skew(*sides[0]) > 0 # negative first skew angle
rotation_angle = (abs(avg_skew - 90) if rotate_right else avg_skew - 90)

upw = lambda s: int(round(s * float(l_width) / s_width)) # scale up width
uph = lambda s: int(round(s * float(l_height) / s_height)) # scale up height

rot_mat = cv2.getRotationMatrix2D(center = (upw(center_x), uph(center_y)), angle = rotation_angle, scale = 1)
rotated_img = cv2.warpAffine(src = img, M = rot_mat, dsize = (l_width,l_height), flags = cv2.INTER_CUBIC)

wid, hei = [abs(tr[0] - tl[0]), abs(bl[1] - tl[1])]
crop = cv2.getRectSubPix(image = rotated_img, patchSize = (upw(wid), uph(hei)), center = (upw(center_x), uph(center_y)))
cv2.imwrite("cropped", crop)
#cv2.waitKey(0)

113